In [43]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
os.environ["OPENAI_API_KEY"]= os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [45]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

print(llm)

client=<openai.resources.chat.completions.Completions object at 0x000001C6EE16EEA0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C6EE16EFD0> root_client=<openai.OpenAI object at 0x000001C6EE20CF50> root_async_client=<openai.AsyncOpenAI object at 0x000001C6F51C8B90> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [46]:
llm.invoke("What is Agentic AI?")

AIMessage(content='Agentic AI refers to artificial intelligence systems designed with agency, meaning they have the capacity to make autonomous decisions, pursue goals, and interact dynamically with their environment. These systems are often characterized by their ability to perceive their surroundings, make decisions based on those perceptions, and take actions to achieve specific objectives, sometimes even adapting their strategies based on feedback and new information.\n\nAgentic AI can range from simple autonomous systems, like basic robotics, to more complex entities capable of sophisticated decision-making processes. The design of such AI often involves elements of machine learning, decision theory, and sometimes even reinforcement learning, where the system learns from experience in a trial-and-error manner to optimize its performance over time.\n\nThe concept of agency in AI also raises important ethical and safety considerations, as systems with autonomous decision-making capa

In [47]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert AI engineer. Provide me answer based on the question."),
        ("user","{input}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI engineer. Provide me answer based on the question.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [48]:
chain = prompt | llm

response = chain.invoke({"input": "Can you tell me about Langsmith"})

In [49]:
print(response)

content='Langsmith is a suite of developer tools designed to enhance the development, testing, and deployment of applications utilizing Language Models (LLMs). It integrates with LLM applications, such as those built using LangChain, and offers features that streamline various stages of the LLM development lifecycle.\n\nKey components of Langsmith include:\n\n1. **Tracer**: This tool facilitates the tracking and debugging of processes within a chain or agent, enhancing transparency and accuracy.\n\n2. **Evaluation Framework**: Langsmith provides a framework for assessing LLM outputs, enabling comprehensive evaluation through LLM-assisted methods, labeled datasets, and preferred scoring metrics.\n\n3. **Dataset Handling**: It supports the efficient management of datasets for LLM application testing and evaluation.\n\n4. **Production Monitoring**: Real-time monitoring capabilities allow developers to keep track of their LLM applications in production, ensuring performance consistency and

In [50]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt| llm | output_parser

response = chain.invoke({"input": "Can you tell me about Langsmith"})
print(response)

Langsmith is a platform developed by LangChain, designed to facilitate the testing, debugging, and monitoring of applications built using large language models (LLMs). It provides extensive tools for developers to optimize their use of these models in applications, focusing on enhancing both performance and reliability.

Key features of Langsmith include:

1. **Testing and Debugging**: Langsmith offers capabilities to thoroughly test language models in various scenarios, enabling developers to identify and fix issues that may arise in LLM behavior.

2. **Monitoring**: It provides insights into how language models are performing in real-time within applications, allowing developers to track key metrics and ensure the models are operating as expected.

3. **Feedback Loops**: The platform helps establish feedback mechanisms to continuously improve model performance based on end-user interactions and other data inputs.

4. **Fine-tuning and Optimization**: Langsmith facilitates ways to fin

In [51]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser()

prompt = PromptTemplate(
    template = "Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables= ["query"],
    partial_variables= {"format_instructions":output_parser.get_format_instructions()}

)

chain = prompt| llm | output_parser

response = chain.invoke({"query": "Can you tell me about Langsmith"})
print(response)

{'name': 'Langsmith', 'overview': 'Langsmith is a platform or toolset designed to facilitate software development and enhance developer productivity. It often includes features such as code management, collaboration tools, and integrations with various development environments.', 'features': ['Code management', 'Collaboration tools', 'Integration with development environments', 'Enhanced developer productivity'], 'use_cases': ['Software development', 'Team collaboration', 'Project management'], 'benefits': ['Streamlined development process', 'Improved team communication', 'Efficient code management']}


### RAG


In [52]:
from langchain_community.document_loaders import WebBaseLoader


In [53]:
loader = WebBaseLoader("https://python.langchain.com/v0.1/docs/modules/chains/")
document = loader.load()
document

[Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/modules/chains/', 'title': 'Chains | 🦜️🔗 LangChain', 'description': 'Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with LCEL.', 'language': 'en'}, page_content="\n\n\n\n\nChains | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentThis is documentation for LangChain v0.1, which is no longer actively maintained. Check out the docs for the latest version here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1Latestv0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchModel I/OPromptsChat modelsLLMsOutput parsersRetrievalDocument loadersText splittersEmbedding modelsVector storesRetrieversIndexingCompositionToolsAgentsChainsMoreComponentsThis is documentation for LangChain v0.1, which is no longer actively maintained.For 

In [54]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
docs = text_splitter.split_documents(document)
docs

[Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/modules/chains/', 'title': 'Chains | 🦜️🔗 LangChain', 'description': 'Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with LCEL.', 'language': 'en'}, page_content='Chains | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/modules/chains/', 'title': 'Chains | 🦜️🔗 LangChain', 'description': 'Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with LCEL.', 'language': 'en'}, page_content="Skip to main contentThis is documentation for LangChain v0.1, which is no longer actively maintained. Check out the docs for the latest version here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1Latestv0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemp

In [55]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


In [56]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs,embeddings)



In [57]:
query = "This application will translate text from English to another language"

result = vectorstore.similarity_search(query)
result

[Document(id='297cfd10-0b27-416d-af40-f9c27254b3ed', metadata={'source': 'https://python.langchain.com/v0.1/docs/modules/chains/', 'title': 'Chains | 🦜️🔗 LangChain', 'description': 'Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with LCEL.', 'language': 'en'}, page_content="Skip to main contentThis is documentation for LangChain v0.1, which is no longer actively maintained. Check out the docs for the latest version here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1Latestv0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchModel I/OPromptsChat modelsLLMsOutput parsersRetrievalDocument loadersText splittersEmbedding modelsVector storesRetrieversIndexingCompositionToolsAgentsChainsMoreComponentsThis is documentation for LangChain v0.1, which is no longer actively maintained.For the c

In [58]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
      """ 
    Answer the following question based on the provided context.
    <context>
    {context}
    </context>
    

   """

)

In [59]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)